In [0]:
import torch  
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
DEVICE 

device(type='cuda')

In [0]:
## 하이퍼 파라미터 설정 
EPOCHS = 50
BATCH_SIZE = 64

In [0]:
##fashion mnist 데이터 셋 불러오기 
image_download_url = "/content/gdrive/My Drive/Colab Notebooks/pytorch/cnn"
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        root = image_download_url,
        train = True,
        download = True,
    
         transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size = BATCH_SIZE,
    shuffle = True
)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        root = image_download_url,
        train = False,
        transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE,
    shuffle=True
)

In [0]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1,out_channels=10,kernel_size=5)
    self.conv2 = nn.Conv2d(in_channels=10,out_channels=20,kernel_size=5)
    self.drop = nn.Dropout2d() ## conv2 거치고 나온것 dropout하기 
    ## dropout해서 나온 것들 이제 일반 신경망 거치기 
    self.fc1 = nn.Linear(in_features=320,out_features=50)
    self.fc2 = nn.Linear(in_features=50,out_features=10) ## 최종 class 는 10개임으로 
  
  def forward(self,x):
    x = F.relu(F.max_pool2d(self.conv1(x),2)) ## 2== kernel 크기 
    x= F.relu(F.max_pool2d(self.conv2(x),2)) 

    ## 현재 x의 형태는 2차원 모양 
    x = x.view(-1,320) ## -1 -> 남은 차원 모두 , 320 = x가 가진 원소개수 

    ##이제 1차원 특징들을 입력받아 class로 분류하는 신경망 구축 
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x =self.fc2(x)

    return F.log_softmax(input=x,dim=1)



In [0]:

model = CNN().to(DEVICE)
optimizer = optim.SGD(params=model.parameters(),lr=0.01,momentum=0.5)


In [0]:
def train (model, train_loader,optimizer,epoch):
  model.train() ## 모델 트레인으로 바꾸고 

  for batch_idx,(data,target) in enumerate(train_loader):
    data, target = data.to(DEVICE) ,target.to(DEVICE)## CUDA로 바꾸고 
  
    optimizer.zero_grad() ## 기울기 0으로 초기화 
    output = model(data)
    loss = F.cross_entropy(output,target)
    loss.backward()
    optimizer.step()

    if(batch_idx %200 ==0):
      print('train epoch : {} [{}/{} ({:.0f}%)]\tLoss : {:.6f}'.format(epoch,batch_idx*len(data),len(train_loader.dataset),100.*batch_idx / len(train_loader),loss.item()))


def evaluate(model, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():##평가 과정에서는 기울기 계산 필요없음 
    for data, target in test_loader:
      data,target = data.to(DEVICE),target.to(DEVICE)
      output = model(data)

      ## 배치 오차 합산 
      test_loss += F.cross_entropy(output,target,reduction='sum').item()

      ## 가장 높은 값ㅇ르 가진 인덱스가 바로 예측 값 
      ## output.max() -> 2개의 값 출력 , 가장 큰 값과 인덱스 
      ## .eq() -> 값이 일치하는지 확인하는 것 
      pred = output.max(1,keepdim=True)[1]
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  test_accuracy = 100.*correct/len(test_loader.dataset)
  return test_loss, test_accuracy





In [39]:
for epoch in range(1,EPOCHS+1):
  train(model,train_loader,optimizer,epoch)
  test_loss ,test_accuracy = evaluate(model,test_loader)
  print('[{}] test loss : {:.4f}, accuracy : {:.2f}%'.format(epoch,test_loss,test_accuracy))

train epoch : 1 [0/60000 (0%)]	Loss : 2.305283
train epoch : 1 [12800/60000 (21%)]	Loss : 1.209217
train epoch : 1 [25600/60000 (43%)]	Loss : 0.922607
train epoch : 1 [38400/60000 (64%)]	Loss : 0.812042
train epoch : 1 [51200/60000 (85%)]	Loss : 0.862138
[1] test loss : 0.6479, accuracy : 75.01%
train epoch : 2 [0/60000 (0%)]	Loss : 0.797771
train epoch : 2 [12800/60000 (21%)]	Loss : 0.963487
train epoch : 2 [25600/60000 (43%)]	Loss : 0.704669
train epoch : 2 [38400/60000 (64%)]	Loss : 0.676444
train epoch : 2 [51200/60000 (85%)]	Loss : 0.630311
[2] test loss : 0.5503, accuracy : 78.72%
train epoch : 3 [0/60000 (0%)]	Loss : 0.534949
train epoch : 3 [12800/60000 (21%)]	Loss : 0.545371
train epoch : 3 [25600/60000 (43%)]	Loss : 0.604474
train epoch : 3 [38400/60000 (64%)]	Loss : 0.503509
train epoch : 3 [51200/60000 (85%)]	Loss : 0.546364
[3] test loss : 0.5018, accuracy : 81.07%
train epoch : 4 [0/60000 (0%)]	Loss : 0.692870
train epoch : 4 [12800/60000 (21%)]	Loss : 0.661247
train epoc